In [2]:

#---------------------------------------------------------------------
# Importing Libraries
print('---------------------------------------------------------------------------------')
print('Importing Libraries')
print('---------------------------------------------------------------------------------')
print('')

import numpy
import time
import tensorflow as tf
import os
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import math
from sklearn.ensemble import RandomForestClassifier
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from imblearn.over_sampling import RandomOverSampler
import shap
from scipy.special import softmax
np.random.seed(0)
from sklearn.model_selection import train_test_split
import sklearn
#---------------------------------------------------------------------
# Defining metric equations

print('---------------------------------------------------------------------------------')
print('Defining Metric Equations')
print('---------------------------------------------------------------------------------')
print('')
def print_feature_importances_shap_values(shap_values, features):
    '''
    Prints the feature importances based on SHAP values in an ordered way
    shap_values -> The SHAP values calculated from a shap.Explainer object
    features -> The name of the features, on the order presented to the explainer
    '''
    # Calculates the feature importance (mean absolute shap value) for each feature
    importances = []
    for i in range(shap_values.values.shape[1]):
        importances.append(np.mean(np.abs(shap_values.values[:, i])))
    # Calculates the normalized version
    importances_norm = softmax(importances)
    # Organize the importances and columns in a dictionary
    feature_importances = {fea: imp for imp, fea in zip(importances, features)}
    feature_importances_norm = {fea: imp for imp, fea in zip(importances_norm, features)}
    # Sorts the dictionary
    feature_importances = {k: v for k, v in sorted(feature_importances.items(), key=lambda item: item[1], reverse = True)}
    feature_importances_norm= {k: v for k, v in sorted(feature_importances_norm.items(), key=lambda item: item[1], reverse = True)}
    # Prints the feature importances
    for k, v in feature_importances.items():
        print(f"{k} -> {v:.4f} (softmax = {feature_importances_norm[k]:.4f})")


def ACC(TP,TN,FP,FN):
    Acc = (TP+TN)/(TP+FP+FN+TN)
    return Acc
def ACC_2 (TP, FN):
    ac = (TP/(TP+FN))
    return ac
def PRECISION(TP,FP):
    Precision = TP/(TP+FP)
    return Precision
def RECALL(TP,FN):
    Recall = TP/(TP+FN)
    return Recall
def F1(Recall, Precision):
    F1 = 2 * Recall * Precision / (Recall + Precision)
    return F1
def BACC(TP,TN,FP,FN):
    BACC =(TP/(TP+FN)+ TN/(TN+FP))*0.5
    return BACC
def MCC(TP,TN,FP,FN):
    MCC = (TN*TP-FN*FP)/(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))**.5)
    return MCC
def AUC_ROC(y_test_bin,y_score):
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    auc_avg = 0
    counting = 0
    for i in range(n_classes):
      fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
      auc_avg += auc(fpr[i], tpr[i])
      counting = i+1
    return auc_avg/counting

def oversample(X_train, y_train):
    oversample = RandomOverSampler(sampling_strategy='minority')
    # Convert to numpy and oversample
    x_np = X_train.to_numpy()
    y_np = y_train.to_numpy()
    x_np, y_np = oversample.fit_resample(x_np, y_np)

    # Convert back to pandas
    x_over = pd.DataFrame(x_np, columns=X_train.columns)
    y_over = pd.Series(y_np)
    return x_over, y_over


###########################################################################################################################################################################################################################################################################################################################
print('---------------------------------------------------------------------------------')
print('DNN DeepLIFT CICIDS GLOBAL')
print('---------------------------------------------------------------------------------')
Trial_1 = [
' ACK Flag Count',
' URG Flag Count',
' PSH Flag Count',
' SYN Flag Count',
'Init_Win_bytes_forward',
' Flow Duration',
'Fwd IAT Total',
' Idle Min',
' Destination Port',
'Bwd IAT Total',
' Idle Max',
' Fwd IAT Std',
' Avg Bwd Segment Size',
' Bwd Packet Length Mean',
' Average Packet Size',
' Flow IAT Std',
' Flow IAT Max',
'FIN Flag Count',
' Init_Win_bytes_backward',
'Bwd Packet Length Max'
]

Trial_2 = [
' PSH Flag Count',
' ACK Flag Count',
' URG Flag Count',
' Fwd IAT Max',
'Fwd IAT Total',
' Flow Duration',
' Idle Min',
'Bwd IAT Total',
'Idle Mean',
' Destination Port',
' Avg Bwd Segment Size',
' Idle Max',
' Max Packet Length',
' Flow IAT Max',
' Fwd IAT Std',
' Init_Win_bytes_backward',
' Flow IAT Std',
' Bwd Packet Length Mean',
' SYN Flag Count',
'FIN Flag Count'
]

Trial_3 = [
' PSH Flag Count',
' ACK Flag Count',
'Fwd IAT Total',
' Flow IAT Max',
'Init_Win_bytes_forward',
' Bwd Packet Length Mean',
' URG Flag Count',
' Idle Min',
'Bwd IAT Total',
' Idle Max',
'Bwd Packet Length Max',
'Fwd PSH Flags',
' Avg Bwd Segment Size',
'Idle Mean',
' Fwd IAT Max',
' Packet Length Std',
' Flow Duration',
' Average Packet Size',
' Destination Port',
' Packet Length Mean'
]

Intersection123 = set(Trial_1) & set(Trial_2) & set(Trial_3)

Total = len(Trial_1)

RF_stab = len(Intersection123)/Total
print(RF_stab)

print('---------------------------------------------------------------------------------')
print('DNN IG CICIDS GLOBAL')
print('---------------------------------------------------------------------------------')


Trial_1 = [
' min_seg_size_forward',
 ' ACK Flag Count',
 ' URG Flag Count',
 ' PSH Flag Count',
 'Init_Win_bytes_forward',
 ' Flow Duration',
 ' SYN Flag Count',
 ' Destination Port',
 'Bwd IAT Total',
 ' Avg Bwd Segment Size',
 ' Idle Max',
 ' Packet Length Mean',
 ' Average Packet Size',
 'Fwd IAT Total',
 ' Fwd IAT Std',
 ' Idle Min',
 ' Avg Fwd Segment Size',
 ' Init_Win_bytes_backward',
 ' Flow IAT Max',
 ' Bwd Packet Length Mean'
]

Trial_2 = [
' min_seg_size_forward',
 ' ACK Flag Count',
 ' Flow Duration',
 'Fwd IAT Total',
 ' Fwd IAT Max',
 ' URG Flag Count',
 'Idle Mean',
 'Bwd IAT Total',
 ' Idle Min',
 ' Avg Bwd Segment Size',
 ' Flow IAT Max',
 ' Max Packet Length',
 ' Destination Port',
 ' PSH Flag Count',
 ' Idle Max',
 ' Flow IAT Std',
 ' Fwd IAT Std',
 ' Init_Win_bytes_backward',
 'FIN Flag Count',
 ' Average Packet Size'
]
Trial_3 =[
' min_seg_size_forward',
 ' Bwd Packet Length Mean',
 ' ACK Flag Count',
 'Fwd IAT Total',
 ' Flow IAT Max',
 'Bwd IAT Total',
 ' Idle Max',
 ' Idle Min',
 ' Avg Bwd Segment Size',
 ' Packet Length Mean',
 ' PSH Flag Count',
 ' Destination Port',
 ' Packet Length Variance',
 ' Flow IAT Std',
 ' URG Flag Count',
 'Bwd Packet Length Max',
 ' Bwd IAT Max',
 ' Fwd IAT Max',
 ' Packet Length Std',
 ' Bwd Packet Length Std'
]
Intersection123 = set(Trial_1) & set(Trial_2) & set(Trial_3)

Total = len(Trial_1)

RF_stab = len(Intersection123)/Total
print(RF_stab)


print('---------------------------------------------------------------------------------')
print('DNN LRP CICIDS GLOBAL')
print('---------------------------------------------------------------------------------')


Trial_1 = [
' min_seg_size_forward',
 ' ACK Flag Count',
 ' URG Flag Count',
 'Init_Win_bytes_forward',
 ' PSH Flag Count',
 ' Flow Duration',
 ' SYN Flag Count',
 ' Destination Port',
 'Bwd IAT Total',
 ' Avg Bwd Segment Size',
 ' Idle Max',
 ' Idle Min',
 'Fwd IAT Total',
 ' Average Packet Size',
 ' Bwd Packet Length Mean',
 ' Fwd IAT Std',
 ' Flow IAT Max',
 ' Packet Length Mean',
 ' Init_Win_bytes_backward',
 ' Flow IAT Std'
]

Trial_2 = [

' min_seg_size_forward',
 ' ACK Flag Count',
 ' Flow Duration',
 ' Fwd IAT Max',
 'Fwd IAT Total',
 ' PSH Flag Count',
 ' URG Flag Count',
 'Idle Mean',
 ' Idle Min',
 ' Avg Bwd Segment Size',
 'Bwd IAT Total',
 ' Flow IAT Max',
 ' Destination Port',
 ' Max Packet Length',
 ' Idle Max',
 ' Flow IAT Std',
 ' Fwd IAT Std',
 ' Init_Win_bytes_backward',
 ' Bwd Packet Length Mean',
 ' Average Packet Size'

]
Trial_3 =[
' min_seg_size_forward',
 ' ACK Flag Count',
 ' Bwd Packet Length Mean',
 'Fwd IAT Total',
 ' Flow IAT Max',
 'Bwd IAT Total',
 ' Idle Min',
 ' Idle Max',
 ' PSH Flag Count',
 ' Packet Length Mean',
 ' Avg Bwd Segment Size',
 ' Flow IAT Std',
 ' URG Flag Count',
 ' Packet Length Variance',
 ' Destination Port',
 'Bwd Packet Length Max',
 'Init_Win_bytes_forward',
 ' Fwd IAT Max',
 ' Bwd IAT Max',
 ' Packet Length Std'
]
Intersection123 = set(Trial_1) & set(Trial_2) & set(Trial_3)

Total = len(Trial_1)

RF_stab = len(Intersection123)/Total
print(RF_stab)

---------------------------------------------------------------------------------
Importing Libraries
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
Defining Metric Equations
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
DNN DeepLIFT CICIDS GLOBAL
---------------------------------------------------------------------------------
0.6
---------------------------------------------------------------------------------
DNN IG CICIDS GLOBAL
---------------------------------------------------------------------------------
0.55
---------------------------------------------------------------------------------
DNN LRP CICIDS GLOBAL
---------------------------------------------------------------------------------
0.65
